## 자동평가 실습
### 활용 데이터: sample.csv

### 실습 개요
이 실습에서는 sample.csv 파일에 포함된 두 개의 텍스트 칼럼에 대해 자동평가 하는 방법을 실습합니다.


### 실습 절차
1. 라이브러리 설치 및 임포트: 필요한 라이브러리 (pandas, nltk, sacrebleu)를 설치하고 임포트합니다.
2. CSV 파일 로드: pandas를 사용하여 주어진 CSV 파일을 읽고 DataFrame으로 변환합니다.
3. 주어진 두 텍스트에 대해 각각 METEOR, LASER, chrF, BERTScore를 계산합니다. (이때 bert토크나이저는 klue/roberta-base모델을 활용합니다.)
4. 결과 저장 및 비교: 두 방법으로 계산된 BLEU 점수를 각각 새로운 열에 저장하고, 결과를 비교 분석합니다.
5. 최종 결과 저장: 계산된 BLEU 및 SacreBLEU 점수가 포함된 DataFrame을 metrics_results.csv 파일로 저장합니다.

In [ ]:
!pip install datasets sacrebleu bert_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
from datasets import load_metric
# from laserembeddings import Laser
# import evaluate

In [11]:
# mymetric = load_metric('meteor')
# chrf = load_metric('chrf')
# bertscore = load_metric("bertscore")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for meteor contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/meteor/meteor.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [12]:
mymetric

Metric(name: "meteor", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Computes METEOR score of translated segments against one or more references.
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    alpha: Parameter for controlling relative weights of precision and recall. default: 0.9
    beta: Parameter for controlling shape of penalty as a function of fragmentation. default: 3
    gamma: Relative weight assigned to fragmentation penalty. default: 0.5
Returns:
    'meteor': meteor score.
Examples:

    >>> meteor = datasets.load_metric('meteor')
    >>> predictions = ["It is a guide to action which ensures that the military always obeys the commands of the party"]
    >>> references = ["It is a gu

In [7]:
df = pd.read_csv('/content/drive/MyDrive/NLP 실습/1 18/sample.csv')

In [8]:
df.head(2)

,sentence1,sentence2
0,"억제 돌연변이 중 NAFM733은 0.1mg/ml로 γPGA를 생성했는데, 이는 다...","응답자의 일상 활동 뿐만 아니라 사회경제적, 개체군통계학적, 직업적 특성에 초점을 ..."
1,"총 알칼리도, 총 암모늄, tVFA와 pH의 농도는 모넨신 사료를 투여한 젖소의 거...","슬라이드를 1,200rpm에서 5분 동안 원심분리하여 건조하고 준비가 될 때까지 어..."


In [13]:
df['meteor'] = df.apply(lambda x: mymetric.compute(predictions=[x['sentence1']], references=[x['sentence2']])['meteor'], axis=1)

In [15]:
df.head(3)

,sentence1,sentence2,meteor
0,"억제 돌연변이 중 NAFM733은 0.1mg/ml로 γPGA를 생성했는데, 이는 다...","응답자의 일상 활동 뿐만 아니라 사회경제적, 개체군통계학적, 직업적 특성에 초점을 ...",0.055866
1,"총 알칼리도, 총 암모늄, tVFA와 pH의 농도는 모넨신 사료를 투여한 젖소의 거...","슬라이드를 1,200rpm에서 5분 동안 원심분리하여 건조하고 준비가 될 때까지 어...",0.037594
2,미생물학적 위험은 제조되는 치즈의 구체적인 특성에 따라 다르다.,축적된 증거는 장내 미생물 군집이 간암 발병에 중요한 역할을 한다는 것을 나타낸다.,0.039683


In [ ]:
df['chrf'] = df.apply(lambda x: chrf.compute(predictions=[x['sentence1']], references=[[x['sentence2']]])['score'], axis=1)

In [ ]:
df.head(2)

,sentence1,sentence2,meteor,chrf
0,"억제 돌연변이 중 NAFM733은 0.1mg/ml로 γPGA를 생성했는데, 이는 다...","응답자의 일상 활동 뿐만 아니라 사회경제적, 개체군통계학적, 직업적 특성에 초점을 ...",0.055866,3.765400
1,"총 알칼리도, 총 암모늄, tVFA와 pH의 농도는 모넨신 사료를 투여한 젖소의 거...","슬라이드를 1,200rpm에서 5분 동안 원심분리하여 건조하고 준비가 될 때까지 어...",0.037594,5.181223


In [ ]:
df['bertscore_model'] = df.apply(lambda x: bertscore.compute(predictions=[x['sentence1']], references=[x['sentence2']], lang="ko"), axis=1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [16]:
!pip install laserembeddings[ko]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.8/859.8 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 40.4 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.35-py3-none-any.whl size=883964 sha256=7e088ad955db2c9d299995e9be3bbb9cf0d824e07edd2f13a6c3b7423be42e5a
  Stored in directory: /root/.cache/pip/wheels/8e/12/4b/5c9eeed3636a4041c004e859e03429a49105672c7fb09ba6d9
Successfully built sacremoses
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu121
    Unins

In [17]:
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [18]:
from laserembeddings import Laser
import numpy as np
from scipy.spatial.distance import cosine

# LASER 모델 초기화
mylaser = Laser()

In [ ]:
df.head(1)

,sentence1,sentence2,meteor,chrf,bertscore_model
0,"억제 돌연변이 중 NAFM733은 0.1mg/ml로 γPGA를 생성했는데, 이는 다...","응답자의 일상 활동 뿐만 아니라 사회경제적, 개체군통계학적, 직업적 특성에 초점을 ...",0.055866,3.7654,"{'precision': [0.6058641076087952], 'recall': ..."


In [19]:
# 문장 임베딩 생성
df['sentence1_laser_embeddings'] = df.apply(lambda x: mylaser.embed_sentences([x['sentence1']], lang='ko'), axis=1)
df['sentence2_laser_embeddings'] = df.apply(lambda x: mylaser.embed_sentences([x['sentence2']], lang='ko'), axis=1)

In [ ]:
df['sentence1_laser_embeddings'][0]

array([0.01249685, 0.00336465, 0.04854362, ..., 0.0129259 , 0.00667777,
       0.04050501], dtype=float32)

In [ ]:
def cosine_similarity(v1, v2):
    n1 = np.linalg.norm(v1)
    n2 = np.linalg.norm(v2)
    return np.dot(v1, v2.T).squeeze() / n1 / n2

In [ ]:
# 코사인 유사도 계산
df['laser_cosine_similarity2'] = df.apply(lambda x: cosine_similarity(x['sentence1_laser_embeddings'], x['sentence2_laser_embeddings']), axis=1)

In [ ]:
df.head(1)

,sentence1,sentence2,meteor,chrf,bertscore_model,sentence1_laser_embeddings,sentence2_laser_embeddings,laser_cosine_similarity,laser_cosine_similarity2
0,"억제 돌연변이 중 NAFM733은 0.1mg/ml로 γPGA를 생성했는데, 이는 다...","응답자의 일상 활동 뿐만 아니라 사회경제적, 개체군통계학적, 직업적 특성에 초점을 ...",0.055866,3.7654,"{'precision': [0.6058641076087952], 'recall': ...","[[0.012496854, 0.0033646508, 0.048543625, 0.00...","[[0.006937603, 0.009236167, -0.00095284154, 0....",0.542123,0.457877


In [ ]:
df.to_csv('metrics_results.csv', index=False)